<a href="https://colab.research.google.com/github/DushyantKhinchi/Twitter_Sentiment_Analysis_Logistic_Regression/blob/master/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing Google Drive to access Dataset
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Checking if the files are available
!ls "drive/Shared drives/Adv Text Homework/HW2"

'hw2 (1).pdf'   hw2_ids566.ipynb   test_10.txt	 train_90.txt


In [ ]:
#importing dependencies
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from scipy.optimize import fmin_tnc
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#importing training and test data
train = pd.read_csv('drive/Shared drives/Adv Text Homework/HW2/train_90.txt', error_bad_lines=False)
test = pd.read_csv('drive/Shared drives/Adv Text Homework/HW2/test_10.txt', error_bad_lines=False)

b'Skipping line 8836: expected 4 fields, saw 5\n'


In [ ]:
#Take a look at the training data
train.head()  

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,is so sad for my APL frie...
1,2,0,Sentiment140,I missed the New Moon trail...
2,3,1,Sentiment140,omg its already 7:30 :O
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...


In [ ]:
#take a look at test data
test.head()

,ItemID,Sentiment,SentimentSource,SentimentText
0,90001,1,Sentiment140,@CleverMonkeys but AFTER you hit up the Magnol...
1,90002,1,Sentiment140,@CleverMonkeys only seen a boot on a car in mo...
2,90003,1,Sentiment140,@clevertitania Good morning. We have rain and ...
3,90004,1,Sentiment140,@clewis4u91 so glad i'm not there then
4,90005,0,Sentiment140,@ClexIsEpic I MSS YOU! sad cookie


In [ ]:
train.shape

(89988, 4)

In [ ]:
test.shape

(10000, 4)

In [ ]:
# function for removing patterns
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

In [ ]:
# remove twitter handles (@user) from training data
train['tidy_tweet'] = np.vectorize(remove_pattern)(train['SentimentText'], "@[\w]*")

In [ ]:
# remove twitter handles (@user) from test data
test['tidy_tweet'] = np.vectorize(remove_pattern)(test['SentimentText'], "@[\w]*")

In [ ]:
# remove special characters, numbers, punctuations
train['tidy_tweet'] = train['tidy_tweet'].str.replace("[^.a-zA-Z#]", " ")
test['tidy_tweet'] = test['tidy_tweet'].str.replace("[^a.-zA-Z#]", " ")

In [ ]:
# converting to lower case
train['tidy_tweet'] = train['tidy_tweet'].str.lower()
test['tidy_tweet'] = test['tidy_tweet'].str.lower()

In [ ]:
def rm_stopwords(raw_text):
    # remove any stopwords
    return [word for word in raw_text.split() if word not in stopwords.words('english')]

In [ ]:
train['tokens'] = train['tidy_tweet'].apply(rm_stopwords)
test['tokens'] = test['tidy_tweet'].apply(rm_stopwords)

In [ ]:
test['tokens'].head()

0    [hit, magnolia, see, anvil, story, anvil, band...
1    [seen, boot, car, movies, goodthing, vegetaria...
2                      [good, morning., rain, thunder]
3                                               [glad]
4                                   [mss, sad, cookie]
Name: tokens, dtype: object

In [ ]:
# initializing tf-idf vetorizer
tfidf_vectorizer = TfidfVectorizer(max_features=200, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
# TF-IDF feature matrix for training and test set
tfidf_train = tfidf_vectorizer.fit_transform(train['tidy_tweet']).toarray()
tfidf_test = tfidf_vectorizer.transform(test['tidy_tweet']).toarray()
#Check the type of the feature matrix
type(tfidf_train)

numpy.ndarray

#Logistic model - sklearn 

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
seed = 7
kfold =KFold(n_splits=10, random_state=seed, shuffle= True)
model = LogisticRegression()
results = cross_val_score(model, tfidf_train, train['Sentiment'], cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Accuracy: 68.101% (0.474%)


#Logistic model - From scratch

In [ ]:
X = tfidf_train
y = train['Sentiment']
theta = np.zeros((X.shape[1], 1))

In [ ]:
def sigmoid(x):
    # Activation function used to map any real value between 0 and 1
    return 1 / (1 + np.exp(-x))

def net_input(theta, x):
    # Computes the weighted sum of inputs
    return np.dot(x, theta)

def probability(theta, x):
    # Returns the probability after passing through sigmoid
    return sigmoid(net_input(theta, x))

In [ ]:
def cost_function(theta, x, y):
    # Computes the cost function for all the training samples
    m = x.shape[0]
    total_cost = -(1 / m) * np.sum(
        y * np.log(probability(theta, x)) + (1 - y) * np.log(
            1 - probability(theta, x)))
    return total_cost

def gradient(theta, x, y):
    # Computes the gradient of the cost function at the point theta
    m = x.shape[0]
    return (1 / m) * np.dot(x.T, sigmoid(net_input(theta,   x)) - y)

In [ ]:
def fit(x, y, theta):
    opt_weights = fmin_tnc(func=cost_function, x0=theta,
                  fprime=gradient,args=(x, y.values.flatten()))
    return opt_weights[0]
parameters = fit(X, y, theta)

In [ ]:
def predict(x):
    theta = parameters[:, np.newaxis]
    return probability(theta, x)
def accuracy(x, actual_classes, probab_threshold=0.5):
    predicted_classes = (predict(x) >= 
                         probab_threshold).astype(int)
    predicted_classes = predicted_classes.flatten()
    accuracy = np.mean(predicted_classes == actual_classes)
    return accuracy
accuracy(X, y.values.flatten())

0.683091078810508

#10-Fold Cross Validation

In [ ]:
#scores = np.zeros((2,2))
scores = []
cv = KFold(n_splits=10, random_state=42, shuffle=False)
for train_index, test_index in cv.split(X,y.values.flatten()):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    parameters = fit(X_train,y_train,theta)
    a=accuracy(X_test, y_test)
    scores.append(a)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
scores
#The parameters from the 10th CV seem to give the best accuracy so we will use these parameters to get the precision and recall for the test set. 

[0.6307367485276142,
 0.6561840204467163,
 0.6829647738637626,
 0.6888543171463496,
 0.6881875763973775,
 0.6906322924769419,
 0.6865207245249472,
 0.6898544282698078,
 0.6875972438319626,
 0.6931540342298288]

Finding precision and recall on test set

In [ ]:
def accuracy_test(x, actual_classes, probab_threshold=0.5):
    predicted_classes = (predict(x) >= 
                         probab_threshold).astype(int)
    predicted_classes = predicted_classes.flatten()
    return confusion_matrix(actual_classes,predicted_classes)

In [ ]:
x = tfidf_test
y = test['Sentiment']
confusion = accuracy_test(x,y.values.flatten())

In [ ]:
precision = confusion[0][0]/(confusion[0][0]+confusion[0][1])

In [ ]:
precision

0.5129805292061907

In [ ]:
recall = confusion[0][0]/(confusion[0][0]+confusion[1][0])

In [ ]:
recall

0.6381987577639752